# The Neighborhoods in LA #

### 1. Install Libraries ###

In [66]:
import pandas as pd
import requests
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
import geocoder
from geopy.geocoders import Nominatim 

import folium

print("Libraries Imported")

Libraries Imported


### 3. Data Import and Data Cleaning ###

In [7]:
# Link: https://www.kaggle.com/cityofLA/crime-in-los-angeles #
df = pd.read_csv("/Users/cl/Desktop/Crime_Data_2010_2017.csv")

In [8]:
df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [9]:
print (df.columns.values.tolist())

['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area ID', 'Area Name', 'Reporting District', 'Crime Code', 'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code', 'Premise Description', 'Weapon Used Code', 'Weapon Description', 'Status Code', 'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Address', 'Cross Street', 'Location ']


In [10]:
df.dropna(subset = ["Location "], inplace=True)

In [11]:
df['Date Occurred']= pd.to_datetime(df['Date Occurred'])

In [43]:
df = df[(df['Date Occurred'] > '06/01/2017') & (df['Date Occurred'] <= '12/31/2017')]
df

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
1390946,170116244,06/07/2017,2017-06-06,2030,1,Central,138,330,BURGLARY FROM VEHICLE,0344,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,3RD,WALL,"(34.048, -118.2438)"
1392055,170411272,06/12/2017,2017-06-11,1800,4,Hollenbeck,412,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,HUMBOLDT,AVENUE 25,"(34.0797, -118.2183)"
1392266,170411315,06/13/2017,2017-06-13,730,4,Hollenbeck,499,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,ESPERANZA,OLYMPIC,"(34.0188, -118.201)"
1392420,170411456,06/16/2017,2017-06-16,1030,4,Hollenbeck,478,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,FRESNO,ATLANTIC,"(34.0268, -118.2075)"
1392758,170512241,06/27/2017,2017-06-26,1800,5,Harbor,526,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,ISLAND,OPP,"(33.7827, -118.2667)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584311,172116482,09/07/2017,2017-09-07,1005,21,Topanga,2141,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,NaN,IC,Invest Cont,440.0,NaN,NaN,NaN,23800 CALVERT ST,NaN,"(34.1883, -118.6536)"
1584312,172116489,09/07/2017,2017-09-07,1200,21,Topanga,2156,330,BURGLARY FROM VEHICLE,0344 1605,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,6600 TOPANGA CANYON BL,NaN,"(34.1905, -118.6059)"
1584313,172116491,08/30/2017,2017-07-15,1,21,Topanga,2147,649,DOCUMENT FORGERY / STOLEN FELONY,0930 1822,...,NaN,IC,Invest Cont,649.0,NaN,NaN,NaN,20900 SHERMAN WY,NaN,"(34.201, -118.5885)"
1584314,172116498,09/08/2017,2017-09-08,1845,21,Topanga,2136,946,OTHER MISCELLANEOUS CRIME,1402,...,NaN,IC,Invest Cont,946.0,998.0,NaN,NaN,REMMET,WYANDOTTE,"(34.2029, -118.6003)"


In [74]:
df_2 = df[["Date Occurred","Area ID","Area Name","Crime Code Description","Address",'Location ']]

In [77]:
df_2['Longitude'], df_2['Latitude'] = df_2['Location '].str.split(',', 1).str
print([s.replace('()', '') for s in df_2])
df_2.head()

['Date Occurred', 'Area ID', 'Area Name', 'Crime Code Description', 'Address', 'Location ', 'Longitude', 'Latitude']


<ipython-input-77-ad7689015e21>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_3['Longitude'], df_3['Latitude'] = df_3['Location '].str.split(',', 1).str
<ipython-input-77-ad7689015e21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Longitude'], df_3['Latitude'] = df_3['Location '].str.split(',', 1).str


,Date Occurred,Area ID,Area Name,Crime Code Description,Address,Location,Longitude,Latitude
1390946,2017-06-06,1,Central,BURGLARY FROM VEHICLE,3RD,"(34.048, -118.2438)",(34.048,-118.2438)
1392055,2017-06-11,4,Hollenbeck,VEHICLE - STOLEN,HUMBOLDT,"(34.0797, -118.2183)",(34.0797,-118.2183)
1392266,2017-06-13,4,Hollenbeck,VEHICLE - STOLEN,ESPERANZA,"(34.0188, -118.201)",(34.0188,-118.201)
1392420,2017-06-16,4,Hollenbeck,VEHICLE - STOLEN,FRESNO,"(34.0268, -118.2075)",(34.0268,-118.2075)
1392758,2017-06-26,5,Harbor,VEHICLE - STOLEN,ISLAND,"(33.7827, -118.2667)",(33.7827,-118.2667)


In [84]:
df_2['Longitude'] = df_2['Longitude'].str.replace("(","") 

<ipython-input-84-6e791ae3f9a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Longitude'] = df_3['Longitude'].str.replace("(","")


In [82]:
df_2['Latitude'] = df_2['Latitude'].str.replace(")","") 

<ipython-input-82-2d122e48f651>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Latitude'] = df_3['Latitude'].str.replace(")","")


In [85]:
df_2.head()

,Date Occurred,Area ID,Area Name,Crime Code Description,Address,Location,Longitude,Latitude
1390946,2017-06-06,1,Central,BURGLARY FROM VEHICLE,3RD,"(34.048, -118.2438)",34.048,-118.2438
1392055,2017-06-11,4,Hollenbeck,VEHICLE - STOLEN,HUMBOLDT,"(34.0797, -118.2183)",34.0797,-118.2183
1392266,2017-06-13,4,Hollenbeck,VEHICLE - STOLEN,ESPERANZA,"(34.0188, -118.201)",34.0188,-118.201
1392420,2017-06-16,4,Hollenbeck,VEHICLE - STOLEN,FRESNO,"(34.0268, -118.2075)",34.0268,-118.2075
1392758,2017-06-26,5,Harbor,VEHICLE - STOLEN,ISLAND,"(33.7827, -118.2667)",33.7827,-118.2667


In [ ]:
df_2.shape

In [86]:
valuecount = df['Crime Code Description'].value_counts()
valuecount

BATTERY - SIMPLE ASSAULT                                        5432
VEHICLE - STOLEN                                                4799
BURGLARY FROM VEHICLE                                           4445
BURGLARY                                                        3734
THEFT PLAIN - PETTY ($950 & UNDER)                              3693
                                                                ... 
LYNCHING                                                           1
BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM0065       1
PETTY THEFT - AUTO REPAIR                                          1
GRAND THEFT / AUTO REPAIR                                          1
BIKE - ATTEMPTED STOLEN                                            1
Name: Crime Code Description, Length: 118, dtype: int64

### 2. Map of Los Angeles ###

### Data Collection ###

In [100]:
url_la = "http://www.laalmanac.com/communications/cm02_communities.php"
zip_la_url = requests.get(url_la)
zip_la_url

<Response [200]>

In [101]:
zip_la_data = pd.read_html(zip_la_url.text)
zip_la_data

[                                  City/Community   Zip Code(s)
 0                                          Acton         93510
 1                                   Agoura Hills         91301
 2                        Agoura Hills (PO Boxes)         91376
 3                                     Agua Dulce         91390
 4                                       Alhambra  91801, 91803
 ..                                           ...           ...
 638                       Winnetka (Los Angeles)         91306
 639            Winnetka (PO Boxes) (Los Angeles)         91396
 640                 Woodland Hills (Los Angeles)  91364, 91367
 641      Woodland Hills (PO Boxes) (Los Angeles)  91365, 91372
 642  Woodland Hills/Pierce College (Los Angeles)         91371
 
 [643 rows x 2 columns]]

In [103]:
zip_la_data = zip_la_data[0]
zip_la_data

,City/Community,Zip Code(s)
0,Acton,93510
1,Agoura Hills,91301
2,Agoura Hills (PO Boxes),91376
3,Agua Dulce,91390
4,Alhambra,"91801, 91803"
...,...,...
638,Winnetka (Los Angeles),91306
639,Winnetka (PO Boxes) (Los Angeles),91396
640,Woodland Hills (Los Angeles),"91364, 91367"
641,Woodland Hills (PO Boxes) (Los Angeles),"91365, 91372"


In [145]:
zip_la_data = zip_la_data.rename(columns={'Zip Code(s)': 'Zip Code'}, index={'ONE': 'Row_1'})
print(zip_la_data)

                                  City/Community      Zip Code
0                                          Acton         93510
1                                   Agoura Hills         91301
2                        Agoura Hills (PO Boxes)         91376
3                                     Agua Dulce         91390
4                                       Alhambra  91801, 91803
..                                           ...           ...
638                       Winnetka (Los Angeles)         91306
639            Winnetka (PO Boxes) (Los Angeles)         91396
640                 Woodland Hills (Los Angeles)  91364, 91367
641      Woodland Hills (PO Boxes) (Los Angeles)  91365, 91372
642  Woodland Hills/Pierce College (Los Angeles)         91371

[643 rows x 2 columns]


In [146]:
zip_la_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
zip_la_data

,City/Community,Zip_Code
0,Acton,93510
1,Agoura Hills,91301
2,Agoura Hills (PO Boxes),91376
3,Agua Dulce,91390
4,Alhambra,"91801, 91803"
...,...,...
638,Winnetka (Los Angeles),91306
639,Winnetka (PO Boxes) (Los Angeles),91396
640,Woodland Hills (Los Angeles),"91364, 91367"
641,Woodland Hills (PO Boxes) (Los Angeles),"91365, 91372"


In [197]:
# remove rows containing '(PO Boxes)'
zip_la_data = zip_la_data[~zip_la_data['City/Community'].str.contains('\(PO Boxes\)')]
zip_la_data

,City/Community,Zip_Code
0,Acton,93510
1,Agoura Hills,91301
3,Agua Dulce,91390
4,Alhambra,"91801, 91803"
5,Alhambra (Non-Geographic Zip Code Within 91801),91804
...,...,...
636,Windsor Hills (Los Angeles),90043
637,Windsor Square (Los Angeles),"90004, 90005, 90010, 90020"
638,Winnetka (Los Angeles),91306
640,Woodland Hills (Los Angeles),"91364, 91367"


### Data Processing ###

In [198]:
zip_la_data.shape

(543, 2)

In [199]:
zip_la_data.info

<bound method DataFrame.info of                                       City/Community  \
0                                              Acton   
1                                       Agoura Hills   
3                                         Agua Dulce   
4                                           Alhambra   
5    Alhambra (Non-Geographic Zip Code Within 91801)   
..                                               ...   
636                      Windsor Hills (Los Angeles)   
637                     Windsor Square (Los Angeles)   
638                           Winnetka (Los Angeles)   
640                     Woodland Hills (Los Angeles)   
642      Woodland Hills/Pierce College (Los Angeles)   

                       Zip_Code  
0                         93510  
1                         91301  
3                         91390  
4                  91801, 91803  
5                         91804  
..                          ...  
636                       90043  
637  90004, 90005, 9001

In [91]:
pip install arcgis

     |████████████████████████████████| 3.2 MB 2.0 MB/s eta 0:00:01
Processing /Users/cl/Library/Caches/pip/wheels/4d/11/58/7d0a04db6c902ef42b717da2981807529f4922485141ab404f/lerc-0.1.0-py3-none-any.whl
  Using cached ujson-4.0.2-cp38-cp38-macosx_10_14_x86_64.whl (45 kB)
  Using cached python_certifi_win32-1.6-py2.py3-none-any.whl (7.2 kB)
Processing /Users/cl/Library/Caches/pip/wheels/1f/1b/b5/54affbefc8a7e2bdf1da000fc576b8a1c91338f1f327a04f4c/pyshp-2.1.3-py3-none-any.whl
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl (5.7 kB)
  Using cached setuptools_scm-6.0.1-py3-none-any.whl (27 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Using cached ntlm_auth-1.5.0-py2.py3-none-any.whl (29 kB)


  Created wheel for arcgis: filename=arcgis-1.8.5.post3-py2.py3-none-any.whl size=3301530 sha256=85f82d67044019df229d83f657e1445948eb9bab58c7c264caef00eb5b73d858
  Stored in directory: /Users/cl/Library/Caches/pip/wheels/04/0a/a2/9a1fd21c8d8ce82f2baab8b9fe6098c828f25590bc3bb8f57a
Successfully built arcgis
ERROR: python-language-server 0.34.1 has requirement ujson<=1.35; platform_system != "Windows", but you'll have ujson 4.0.2 which is incompatible.
ERROR: python-jsonrpc-server 0.3.4 has requirement ujson<=1.35; platform_system != "Windows", but you'll have ujson 4.0.2 which is incompatible.
  Attempting uninstall: ujson
    Found existing installation: ujson 1.35
    Uninstalling ujson-1.35:
      Successfully uninstalled ujson-1.35
Note: you may need to restart the kernel to use updated packages.


In [137]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()
from arcgis.geoenrichment import Country

In [150]:
def get_x_y_us(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, Los angeles, CA'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [139]:
results = geocode('Los Angeles, CA')
results[0]['location']

{'x': -118.24544999999995, 'y': 34.053570000000036}

In [200]:
geo_coordinates_us = zip_la_data['Zip_Code']    
geo_coordinates_us

0                           93510
1                           91301
3                           91390
4                    91801, 91803
5                           91804
                  ...            
636                         90043
637    90004, 90005, 90010, 90020
638                         91306
640                  91364, 91367
642                         91371
Name: Zip_Code, Length: 543, dtype: object

In [201]:
coordinates_latlng_us = geo_coordinates_us.apply(lambda x: get_x_y_us(x))
coordinates_latlng_us

0       34.46937430000003,-118.20080219999994
1      34.150190000000066,-118.75928499999998
3       34.49227500000006,-118.33201999999994
4       34.08087500000005,-118.14817999999997
5       34.09370000000007,-118.12726999999995
                        ...                  
636    33.990815000000055,-118.33612499999998
637     34.06702300000006,-118.29074769999994
638     34.20390000000003,-118.57539499999996
640    34.178300000000036,-118.60802499999994
642     34.18250500000005,-118.57095999999996
Name: Zip_Code, Length: 543, dtype: object

## Latitude ##

In [202]:
lat_us = coordinates_latlng_us.apply(lambda x: x.split(',')[0])
lat_us

0       34.46937430000003
1      34.150190000000066
3       34.49227500000006
4       34.08087500000005
5       34.09370000000007
              ...        
636    33.990815000000055
637     34.06702300000006
638     34.20390000000003
640    34.178300000000036
642     34.18250500000005
Name: Zip_Code, Length: 543, dtype: object

## Longitude ##

In [203]:
lng_us = coordinates_latlng_us.apply(lambda x: x.split(',')[1])
lng_us

0      -118.20080219999994
1      -118.75928499999998
3      -118.33201999999994
4      -118.14817999999997
5      -118.12726999999995
              ...         
636    -118.33612499999998
637    -118.29074769999994
638    -118.57539499999996
640    -118.60802499999994
642    -118.57095999999996
Name: Zip_Code, Length: 543, dtype: object

In [204]:
la_merged = pd.concat([zip_la_data,lat_us.astype(float), lng_us.astype(float)], axis=1)
la_merged.columns= ['City/Community','Zip_Code','latitude','longitude']
la_merged

,City/Community,Zip_Code,latitude,longitude
0,Acton,93510,34.469374,-118.200802
1,Agoura Hills,91301,34.150190,-118.759285
3,Agua Dulce,91390,34.492275,-118.332020
4,Alhambra,"91801, 91803",34.080875,-118.148180
5,Alhambra (Non-Geographic Zip Code Within 91801),91804,34.093700,-118.127270
...,...,...,...,...
636,Windsor Hills (Los Angeles),90043,33.990815,-118.336125
637,Windsor Square (Los Angeles),"90004, 90005, 90010, 90020",34.067023,-118.290748
638,Winnetka (Los Angeles),91306,34.203900,-118.575395
640,Woodland Hills (Los Angeles),"91364, 91367",34.178300,-118.608025


In [205]:
la_merged.dtypes

City/Community     object
Zip_Code           object
latitude          float64
longitude         float64
dtype: object

In [206]:
la = geocode(address='Los Angeles, CA')[0]
la_lng_coords = la['location']['x']
la_lat_coords = la['location']['y']
la_lng_coords

-118.24544999999995

In [207]:
la_lat_coords

34.053570000000036

## Visualize the Map of Los Angeles, CA ##

In [162]:
# Creating the map of LA
map_LA = folium.Map(location=[la_lat_coords, la_lng_coords], zoom_start=12)
map_LA

# adding markers to map
for latitude, longitude, borough in zip(la_merged['latitude'], la_merged['longitude'], la_merged['City/Community']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_LA)  
    
map_LA

### Venues in Los Angeles ###

In [185]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'CA4LWP3UODKZZTBZNB2I3UVAPNRWEMOZDZYB2O0RFR5R2HG3'
CLIENT_SECRET = 'RAHCZTHIGNRNTKAV5PEK0VGKZLIXQU1NX0VA0T4SXMH3YXFH'

VERSION = '20210420'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CA4LWP3UODKZZTBZNB2I3UVAPNRWEMOZDZYB2O0RFR5R2HG3
CLIENT_SECRET:RAHCZTHIGNRNTKAV5PEK0VGKZLIXQU1NX0VA0T4SXMH3YXFH


In [192]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,  LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()['response'].get('groups',[{}])[0].get('items', [])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [208]:
venues_in_LA = getNearbyVenues(la_merged['City/Community'], la_merged['latitude'], la_merged['longitude'])

Acton
Agoura Hills
Agua Dulce
Alhambra
Alhambra (Non-Geographic Zip Code Within 91801)
Altadena
Americana at Brand & Glendale Galleria (Glendale)
Arcadia
Arleta (Los Angeles)
Arlington Heights (Los Angeles)
Artesia
Athens
Atwater Village (Los Angeles)
Avocado Heights
Azusa
Baldwin Hills (Los Angeles)
Baldwin Park
Bassett
Bel Air Estates (Los Angeles)
Bel Air Estates, Beverly Glen (Los Angeles)
Bell
Bell (Shared Firms)
Bell Canyon
Bell Gardens
Bellflower
Belmont Shore (Long Beach)
Beverly Glen, Bel Air Estates (Los Angeles)
Beverly Hills
Biola University (La Mirada)
Bixby Knolls (Long Beach)
Bouquet Canyon
Box Canyon
Boyle Heights (Los Angeles)
Bradbury
Brentwood (Los Angeles)
Burbank
Burbank (Burbank Studios)
Burbank (Glenoaks)
Burbank (Walt Disney Co)
Burbank Studios (Burbank)
Bureau of the Census (Pomona)
Byzantine-Latino Quarter (Los Angeles)
Calabasas
California Institute of Technology (Pasadena)
California State Univ Long Beach (Long Beach)
California State Univ Northridge (Los An

Los Angeles (West Adams)
Los Angeles (West Beverly)
Los Angeles (West Fairfax)
Los Angeles (West Hills)
Los Angeles (West Los Angeles)
Los Angeles (West Toluca Lake)
Los Angeles (Westchester)
Los Angeles (Westchester PO Boxes)
Los Angeles (Western Wilton)
Los Angeles (Westlake)
Los Angeles (Westwood)
Los Angeles (Wilmington)
Los Angeles (Wilmington PO Boxes)
Los Angeles (Wilshire Center)
Los Angeles (Wilshire Center PO Boxes)
Los Angeles (Wilshire Park)
Los Angeles (Windsor Hills)
Los Angeles (Windsor Square)
Los Angeles (Winnetka)
Los Angeles (Winnetka PO Boxes)
Los Angeles (Woodland Hills)
Los Angeles (Woodland Hills PO Boxes)
Los Angeles Dodgers (Los Angeles)
Los Angeles International Airport, Westchester (Los Angeles)
Los Feliz (Los Angeles)
Los Nietos
Lynwood
Malibu
Malibu (Pepperdine University)
Manhattan Beach
Mar Vista (Los Angeles)
Marina (Long Beach)
Marina del Rey
Marina Peninsula (Los Angeles)
Maywood
Melrose (Los Angeles)
Metler Valley
Mid-City (Los Angeles)
Mid-City West 

In [209]:
venues_in_LA.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Acton,34.469374,-118.200802,Acton Market & Country Store,Grocery Store
1,Acton,34.469374,-118.200802,Acton Community Park,Park
2,Acton,34.469374,-118.200802,TrichStop.com,Health & Beauty Service
3,Acton,34.469374,-118.200802,High Mesa,Food
4,Acton,34.469374,-118.200802,V O's Painting & Renovating Inc,Construction & Landscaping


In [210]:
venues_in_LA.shape

(14223, 5)

### Grouping by Venue Categories ###

In [211]:
venues_in_LA.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
ATM,Westwood (Los Angeles),34.385189,-117.968705,Wells Fargo ATM
Accessories Store,West Hollywood,34.181830,-118.027985,lasc - los angeles sports club
Adult Boutique,Van Nuys/US Purchasing Exchange (Los Angeles),34.151070,-118.444340,69 Adult Toys - Sherman Oaks
Advertising Agency,San Marino,34.121580,-118.106575,LA County DBAs
Airport Service,Burbank Studios (Burbank),34.156265,-118.333985,Airtran Airways
...,...,...,...,...
Wine Shop,Tropico (Glendale),34.172685,-118.139925,Uncorked The Wine Shop
Winery,Pasadena (Shared Firms),34.147450,-118.144270,Bodega Wine Bar
Wings Joint,Wilshire Park (Los Angeles),34.416395,-117.999145,Wingstop


### One Hot Encoding ###

In [212]:
LA_venue_cat = pd.get_dummies(venues_in_LA[['Venue Category']], prefix="", prefix_sep="")
LA_venue_cat

,ATM,Accessories Store,Adult Boutique,Advertising Agency,Airport Service,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
LA_venue_cat['Neighbourhood'] = venues_in_LA['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [LA_venue_cat.columns[-1]] + list(LA_venue_cat.columns[:-1])
LA_venue_cat = LA_venue_cat[fixed_columns]

LA_venue_cat.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Airport Service,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,...,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Venue categories mean value ###

In [214]:
LA_grouped = LA_venue_cat.groupby('Neighbourhood').mean().reset_index()
LA_grouped.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Airport Service,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,...,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Acton,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,Agoura Hills,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,Alhambra,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,Alhambra (Non-Geographic Zip Code Within 91801),0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.022222,0.0,0.0
4,Altadena,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [215]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [216]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

### Top Venues Categories ###

In [218]:
# create a new dataframe for London
neighborhoods_venues_sorted_la = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_la['Neighbourhood'] = LA_grouped['Neighbourhood']

for ind in np.arange(LA_grouped.shape[0]):
    neighborhoods_venues_sorted_la.iloc[ind, 1:] = return_most_common_venues(LA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_la.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Grocery Store,Food,Construction & Landscaping,Pet Store,Health & Beauty Service,Park,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room
1,Agoura Hills,Fast Food Restaurant,ATM,Coffee Shop,Sushi Restaurant,Breakfast Spot,Sports Bar,Burger Joint,Smoke Shop,Café,Shopping Mall
2,Alhambra,Burger Joint,Mexican Restaurant,Gift Shop,Gym,Coffee Shop,Optical Shop,Juice Bar,Sandwich Place,Taco Place,Liquor Store
3,Alhambra (Non-Geographic Zip Code Within 91801),Bakery,Ice Cream Shop,Chinese Restaurant,Burger Joint,Seafood Restaurant,Café,Sushi Restaurant,Thai Restaurant,Grocery Store,Coffee Shop
4,Altadena,Grocery Store,Bank,Dive Bar,Food Truck,Food & Drink Shop,Food,Mexican Restaurant,Men's Store,Fast Food Restaurant,Scenic Lookout


## Model Building ##

In [219]:
#K Means
# cluster the city of LA to 5
k_num_clusters = 5

LA_grouped_clustering = LA_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_la = KMeans(n_clusters=k_num_clusters, random_state=0).fit(LA_grouped_clustering)
kmeans_la

KMeans(n_clusters=5, random_state=0)

In [220]:
# Labelling Clustered Data
kmeans_la.labels_

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [221]:
neighborhoods_venues_sorted_la.insert(0, 'Cluster Labels', kmeans_la.labels_ +1)

In [223]:
la_data = la_merged

la_data = la_data.join(neighborhoods_venues_sorted_la.set_index('Neighbourhood'), on='City/Community')

la_data.head()

,City/Community,Zip_Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,93510,34.469374,-118.200802,2.0,Grocery Store,Food,Construction & Landscaping,Pet Store,Health & Beauty Service,Park,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room
1,Agoura Hills,91301,34.150190,-118.759285,2.0,Fast Food Restaurant,ATM,Coffee Shop,Sushi Restaurant,Breakfast Spot,Sports Bar,Burger Joint,Smoke Shop,Café,Shopping Mall
3,Agua Dulce,91390,34.492275,-118.332020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alhambra,"91801, 91803",34.080875,-118.148180,2.0,Burger Joint,Mexican Restaurant,Gift Shop,Gym,Coffee Shop,Optical Shop,Juice Bar,Sandwich Place,Taco Place,Liquor Store
5,Alhambra (Non-Geographic Zip Code Within 91801),91804,34.093700,-118.127270,2.0,Bakery,Ice Cream Shop,Chinese Restaurant,Burger Joint,Seafood Restaurant,Café,Sushi Restaurant,Thai Restaurant,Grocery Store,Coffee Shop


In [243]:
la_data_nonan = la_data.dropna(subset=['Cluster Labels'])
la_data_nonan.head()

,City/Community,Zip_Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,93510,34.469374,-118.200802,2.0,Grocery Store,Food,Construction & Landscaping,Pet Store,Health & Beauty Service,Park,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room
1,Agoura Hills,91301,34.150190,-118.759285,2.0,Fast Food Restaurant,ATM,Coffee Shop,Sushi Restaurant,Breakfast Spot,Sports Bar,Burger Joint,Smoke Shop,Café,Shopping Mall
4,Alhambra,"91801, 91803",34.080875,-118.148180,2.0,Burger Joint,Mexican Restaurant,Gift Shop,Gym,Coffee Shop,Optical Shop,Juice Bar,Sandwich Place,Taco Place,Liquor Store
5,Alhambra (Non-Geographic Zip Code Within 91801),91804,34.093700,-118.127270,2.0,Bakery,Ice Cream Shop,Chinese Restaurant,Burger Joint,Seafood Restaurant,Café,Sushi Restaurant,Thai Restaurant,Grocery Store,Coffee Shop
7,Altadena,91001,34.185630,-118.133870,2.0,Grocery Store,Bank,Dive Bar,Food Truck,Food & Drink Shop,Food,Mexican Restaurant,Men's Store,Fast Food Restaurant,Scenic Lookout


## Visualize the Clustered Neighbourhood ##

In [226]:
map_clusters_la = folium.Map(location=[la_lat_coords, la_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_data_nonan['latitude'], la_data_nonan['longitude'], la_data_nonan['City/Community'], la_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_la)
        
map_clusters_la

## Examine the Clusters ##

In [228]:
# Cluster 1

la_data_nonan.loc[la_data_nonan['Cluster Labels'] == 1, la_data_nonan.columns[[1] + list(range(5, la_data_nonan.shape[1]))]]

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,91331,Convenience Store,Mexican Restaurant,Grocery Store,Seafood Restaurant,Electronics Store,Mobile Phone Shop,Breakfast Spot,Field,Escape Room,Ethiopian Restaurant
19,91746,Mexican Restaurant,Automotive Shop,Dessert Shop,Donut Shop,Sandwich Place,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Fabric Shop
23,91746,Mexican Restaurant,Automotive Shop,Dessert Shop,Donut Shop,Sandwich Place,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Fabric Shop
30,90201,Mexican Restaurant,Pizza Place,Bank,Grocery Store,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Diner,Video Game Store
42,91304,Chiropractor,Park,Middle Eastern Restaurant,Convenience Store,Mexican Restaurant,Tennis Court,Bar,Fast Food Restaurant,Escape Room,Ethiopian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
573,90032,Mexican Restaurant,Fast Food Restaurant,Seafood Restaurant,ATM,Asian Restaurant,South American Restaurant,Brazilian Restaurant,Shoe Store,Neighborhood,Chinese Restaurant
603,"90002, 90059",Mexican Restaurant,Chinese Restaurant,Burger Joint,Park,Fish Market,Snack Place,Liquor Store,Gas Station,Donut Shop,Fast Food Restaurant
606,90016,Breakfast Spot,Mexican Restaurant,Playground,Café,Yoga Studio,Film Studio,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market
614,91746,Mexican Restaurant,Automotive Shop,Dessert Shop,Donut Shop,Sandwich Place,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Fabric Shop


In [229]:
# Cluster 2

la_data_nonan.loc[la_data_nonan['Cluster Labels'] == 2, la_data_nonan.columns[[1] + list(range(5, la_data_nonan.shape[1]))]]

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,93510,Grocery Store,Food,Construction & Landscaping,Pet Store,Health & Beauty Service,Park,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room
1,91301,Fast Food Restaurant,ATM,Coffee Shop,Sushi Restaurant,Breakfast Spot,Sports Bar,Burger Joint,Smoke Shop,Café,Shopping Mall
4,"91801, 91803",Burger Joint,Mexican Restaurant,Gift Shop,Gym,Coffee Shop,Optical Shop,Juice Bar,Sandwich Place,Taco Place,Liquor Store
5,91804,Bakery,Ice Cream Shop,Chinese Restaurant,Burger Joint,Seafood Restaurant,Café,Sushi Restaurant,Thai Restaurant,Grocery Store,Coffee Shop
7,91001,Grocery Store,Bank,Dive Bar,Food Truck,Food & Drink Shop,Food,Mexican Restaurant,Men's Store,Fast Food Restaurant,Scenic Lookout
...,...,...,...,...,...,...,...,...,...,...,...
636,90043,Fried Chicken Joint,Discount Store,Burger Joint,BBQ Joint,Caribbean Restaurant,Dry Cleaner,Donut Shop,Vegetarian / Vegan Restaurant,Grocery Store,American Restaurant
637,"90004, 90005, 90010, 90020",Korean Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Supermarket,Food Truck,Video Store,Karaoke Bar,Pharmacy,Donut Shop
638,91306,Pizza Place,Grocery Store,Fast Food Restaurant,Convenience Store,American Restaurant,Sandwich Place,Home Service,Empanada Restaurant,English Restaurant,Escape Room
640,"91364, 91367",Bank,Park,Deli / Bodega,Mobile Phone Shop,Convenience Store,Movie Theater,Rental Car Location,Gym / Fitness Center,Men's Store,Chinese Restaurant


In [230]:
# Cluster 3

la_data_nonan.loc[la_data_nonan['Cluster Labels'] == 3, la_data_nonan.columns[[1] + list(range(5, la_data_nonan.shape[1]))]]




,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
167,93535,Residential Building (Apartment / Condo),Yoga Studio,Filipino Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market
493,93535,Residential Building (Apartment / Condo),Yoga Studio,Filipino Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market
635,93535,Residential Building (Apartment / Condo),Yoga Studio,Filipino Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market


In [231]:
# Cluster 4

la_data_nonan.loc[la_data_nonan['Cluster Labels'] == 4, la_data_nonan.columns[[1] + list(range(5, la_data_nonan.shape[1]))]]

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,90895,Grocery Store,Burger Joint,Arts & Crafts Store,Filipino Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
80,93551,Grocery Store,Yoga Studio,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
210,"93532, 93551",Grocery Store,Yoga Studio,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
483,90220,Grocery Store,Accessories Store,Fried Chicken Joint,Yoga Studio,Filipino Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market


In [232]:
# Cluster 5

la_data_nonan.loc[la_data_nonan['Cluster Labels'] == 5, la_data_nonan.columns[[1] + list(range(5, la_data_nonan.shape[1]))]]

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
291,"91324, 91325, 91326",Music Venue,Yoga Studio,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
307,91326,Music Venue,Yoga Studio,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
431,"91324, 91325, 91326",Music Venue,Yoga Studio,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
479,91326,Music Venue,Yoga Studio,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
